In [29]:
import pandas as pd
from census import Census

In [30]:
# Get Variable Code
acs_code_df = pd.read_csv('../data/acs_variable_code.csv')
acs_dict = {}
for i in range(len(acs_code_df)):
    acs_dict[acs_code_df.loc[i,'Variable Name']] = acs_code_df.loc[i,'Variable Code']

var_list = []
for f in options:
    var_list.append(acs_dict{f})

SyntaxError: invalid syntax. Perhaps you forgot a comma? (2171837190.py, line 9)

In [28]:

def get_file_contents(filename):
    """ Given a filename,
        return the contents of that file
    """
    try:
        with open(filename, 'r') as f:
            # It's assumed our file contains a single line,
            # with our API key
            return f.read().strip()
    except FileNotFoundError:
        print("'%s' file not found" % filename)


In [ ]:
api_file = 'census_api_key.txt'
api_key = get_file_contents(api_file)
c = Census(api_key)
# Replace with Input from lat, lng
state_fips = '06'
county_fips = '085'

sc_census = c.acs5.state_county_blockgroup(fields= var_list,
                                      state_fips = state_fips,
                                      county_fips = county_fips,
                                      blockgroup = '*',
                                      year = 2019)


In [ ]:
#### 2. Feature engineering
# Feature Engineering
acs_df = pd.DataFrame(sc_census)
acs_df['geoid'] = sc_df['state'] + sc_df['county'] + sc_df['tract'] + sc_df['block group']


inv_acs_dict = {v: k for k, v in acs_dict.items()}
acs_df.rename(columns=inv_acs_dict, inplace=True)

acs_df.head()